In [1]:
import numpy as np
import os
import operator
import pandas as pd
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, plot, iplot
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

init_notebook_mode(connected=True)

## Loading data

In [2]:
# Loading the data into as a pandas data frame
data = pd.read_csv("./data/ubuntu_support_extract.csv")
data.head(10)

,conversation_id,datetime,from,to,text
0,10-10000,2010-04-17 20:15:00+00:00,fk91,NaN,"Hello, I have a minimal linux system: how can ..."
1,10-10000,2010-04-17 20:15:00+00:00,fk91,NaN,"@Maco: ip is there, thanks :)"
2,10-10000,2010-04-17 20:15:00+00:00,sometux,fk91,ifconfig
3,10-10000,2010-04-17 20:15:00+00:00,sometux,fk91,static or dhcp
4,10-10000,2010-04-17 20:16:00+00:00,fk91,NaN,static
5,10-10000,2010-04-17 20:16:00+00:00,sometux,fk91,look at /etc/interface
6,10-10000,2010-04-17 20:17:00+00:00,fk91,sometux,"/etc/interfaces are not there, its a fli4l"
7,10-10000,2010-04-17 20:17:00+00:00,sometux,fk91,sorry look at /etc/network/interfaces
8,10-10000,2010-04-17 20:18:00+00:00,fk91,sometux,This file isnt there too.
9,10-10000,2010-04-17 20:19:00+00:00,sometux,fk91,i think you have to look in /proc


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3075574 entries, 0 to 3075573
Data columns (total 5 columns):
conversation_id    object
datetime           object
from               object
to                 object
text               object
dtypes: object(5)
memory usage: 117.3+ MB


In [4]:
# Checking number of rows with missing values in each of the columns
data.isnull().sum()

conversation_id          0
datetime                 0
from                    25
to                 1133776
text                     2
dtype: int64

In [5]:
# Removing rows which with missing values in 'text' and 'from' columns only,
# column 'to' can have valid null values in the first msg of the conversation.
data = data.dropna(subset=['text','from']) 
data.shape

(3075547, 5)

In [6]:
# Verifying if rows with null values in 'text' and 'from' columns are removed
data.isnull().sum()

conversation_id          0
datetime                 0
from                     0
to                 1133752
text                     0
dtype: int64

In [7]:
from_users = data['from'].tolist()
to_users = data['to'].tolist()
print(len(set(from_users)), ',', len(set(to_users)))

92495 , 89090


In [8]:
# Total number of unique users
print(len(set(from_users + to_users)))

92519


## Count based approach for selecting agents
A naive count based selection of agents. Rationale is that the users involved in high number of messages/conversations are:
1. either advisors who responded to an issue
2. or enquirers who gained knowledge after querying about an issue

Either way, users involved in both sides of conversation are assumed to have knowledge of the covered topics
'after' a conversation has taken place.

Minimal incremental improvements: 
1. Filter out users who were on the advising side most of the times, based on the initial inquiry in the conversation.
2. Give more weightage to the users who are currently active, based on date time. 
3. Ranking agents based on the average handle/response time based on the time stamps. 
4. The below agent selection mechanism does not use the language used in the conversations to filter out the conversations in a different language. But this can be done in a minimal way by using off the shelf language detectors.

In [9]:
# Computing total incoming and outgoing messages for each user. 
from_user_dist = data.drop(columns=['text','to','datetime']).groupby(['from'])\
                              .size().reset_index(name='outgoing')

from_user_dist = from_user_dist.rename(columns={'from': 'users'})

to_user_dist = data.drop(columns=['text','from','datetime']).groupby(['to'])\
                            .size().reset_index(name='incoming') 
to_user_dist = to_user_dist.rename(columns={'to': 'users'})

In [10]:
# Sorting the agents based on the volume of outgoing/incoming messages.
incoming_outgoing = pd.merge(from_user_dist, to_user_dist, on='users')\
                    .sort_values(ascending=False, by='outgoing')
incoming_outgoing.head(15)

,users,outgoing,incoming
349,ActionParsnip,55125,27053
6001,Dr_Willis,36626,16590
51341,ikonia,30715,13762
43183,edbian,21307,9511
54888,jrib,18491,11398
44083,erUSUL,17653,9788
33104,bazhang,17512,6701
38388,coz_,14329,5404
13215,Jordan_U,13537,8821
81069,theadmin,13047,6662


## Topic based approach for selecting agents

The above quantitative approach does not take into account the topics/knowledge coverage, 
and the distribution of topics in the conversations. If conversations cover 3 topics in a 
distribution of 60:20:20, a good approach will be that the top 15 agents should be distributed across 
the 3 topics in a similar way, i.e., 9:3:3

Topics can be seen as clusters, where algorithms like LDA (~soft clustering approach) can be utilized to uncover themes being discussed in the conversations.

This approach first identifies n number of topics in the full dataset, and calculate the coverage of each topic in each message, using LDA. LDA identifies topics as a cluster of tokens. N more columns are added to the dataset where each column represents a topic, and holds a % value which is the topic coverage of the message. This final dataset can be queried in a number of ways to make a topic based selection of users.

In [11]:
def plot_frequent_n_words(n, word_count_dict):
    sorted_word_count = sorted(word_count_dict.items(), key=operator.itemgetter(1), reverse=True)
    word_count = sorted_word_count[:n+1]
    trace1 = go.Bar(
      x = [i[0] for i in word_count], 
      y = [i[1] for i in word_count], 
      marker = dict(color='blue'))

    data = [trace1]
    layout = go.Layout(
      title= "Ngrams Frequency", 
      xaxis= dict(
      title= "Ngrams"), 
      yaxis=dict(title="Count")
    )
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='jupyter-styled_bar')

In [12]:
def print_topics(model, vectorizer, num_ngrams):
    ngrams = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(", ".join([ngrams[i]
                        for i in topic.argsort()[:-num_ngrams - 1:-1]]))

In [13]:
data_sample = data.sample(n=10000)
data_sample.shape

(10000, 5)

In [14]:
# TODO
# Improvement: text pre-processing. Removal of stopwords, adjectives, adverbs etc. 
# which are less likely to represent the topic in a conversation. 

In [15]:
count_vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 3))
ngrams_cv = count_vectorizer.fit_transform(data_sample['text'])
# Ngrams and their count
ngrams = count_vectorizer.get_feature_names()
counts = ngrams_cv.toarray().sum(axis=0)        

In [16]:
plot_frequent_n_words(20, dict(zip(ngrams, counts)))

In [17]:
# LDA parameters 
number_topics = 5
topic_ngrams = 20

# Learning a topic model on the message texts
lda = LDA(n_components=number_topics)
lda.fit(ngrams_cv)

# Print ngrams in each topic identified by LDA, each topic would have its own weights 
# over different ngrams.
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, topic_ngrams)

Topics found via LDA:

Topic #0:
ubuntu, http, ok, hi, know, install, like, run, 10, hello, want, help, just, com, file, installed, thanks, does, set, 04

Topic #1:
ubuntu, use, don, install, com, know, using, try, just, paste, http, need, ubuntu com, linux, want, help, windows, boot, files, like

Topic #2:
thanks, ubuntu, 10, work, does, did, know, use, 04, don, like, install, doesn, need, help, just, desktop, try, error, run

Topic #3:
ubuntu, sudo, 10, apt, just, install, sudo apt, help, file, like, use, way, ok, problem, try, work, apt install, installed, time, yeah

Topic #4:
just, ubuntu, install, use, yes, windows, problem, want, file, boot, try, need, using, know, does, think, good, make, question, ok


In [18]:
#transform text messages into their respective topic distributions
message_lda_topics_vectors = lda.transform(ngrams_cv).tolist()

# Print topic distribution for 5 sample messages.
for n in range(5):    
    topic_pr = message_lda_topics_vectors[n]    
    print("msg: {} topic: {}\n".format(n, topic_pr))

msg: 0 topic: [0.7986395207574454, 0.05040180389808633, 0.05010753007197424, 0.05044916790748028, 0.05040197736501366]

msg: 1 topic: [0.8841835316522276, 0.029126751953631545, 0.028851145405423263, 0.028789080759548, 0.029049490229169483]

msg: 2 topic: [0.7968212999978639, 0.05079181805026459, 0.05112566297465248, 0.05067560621505579, 0.050585612762163314]

msg: 3 topic: [0.2, 0.2, 0.2, 0.2, 0.2]

msg: 4 topic: [0.028947397809676315, 0.028735322722713327, 0.8844293198347631, 0.028753151933400795, 0.0291348076994465]



In [19]:
topics_df = pd.DataFrame(message_lda_topics_vectors, columns = ['Topic 1', 'Topic 2','Topic 3','Topic 4','Topic 5'])
topics_df.head(10)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
0,0.798640,0.050402,0.050108,0.050449,0.050402
1,0.884184,0.029127,0.028851,0.028789,0.029049
2,0.796821,0.050792,0.051126,0.050676,0.050586
3,0.200000,0.200000,0.200000,0.200000,0.200000
4,0.028947,0.028735,0.884429,0.028753,0.029135
5,0.020118,0.919359,0.020144,0.020097,0.020283
6,0.597772,0.100399,0.100429,0.100487,0.100913
7,0.100946,0.100471,0.101304,0.101999,0.595280
8,0.919096,0.020256,0.020144,0.020235,0.020268
9,0.006483,0.006779,0.006543,0.006528,0.973667


In [20]:
# Adding a column 'message_id' to use it for join with the topic vectors
data_sample['message_id'] = range(1, len(data_sample) + 1)
data_sample.head(10)

,conversation_id,datetime,from,to,text,message_id
2835298,8-46142,2010-10-25 02:11:00+00:00,Kai696,NaN,anyone spare a moment ^^ ?,1
2395598,6-31675,2011-07-17 19:49:00+00:00,TheFuzzball,NaN,macs That should be installed by default...,2
2158469,5-45215,2010-09-03 08:54:00+00:00,chris_osx,nightshade,why are you asking that on #ubuntu?,3
1097685,22-1223,2010-08-11 04:47:00+00:00,guest99,profxavier,it is.,4
1069991,21-2133,2010-01-07 22:33:00+00:00,Deathspike,Mortal_,"No, the tool doesnt even exist on my system.",5
2697091,7-9525,2011-04-23 23:40:00+00:00,ceo,erUSUL,am need install inventory.bin,6
1523716,3-78839,2010-04-15 01:14:00+00:00,MP1,NaN,Hello,7
2228555,51-114,2010-09-03 05:35:00+00:00,undecim,skjoedt,no problem,8
2923036,9-11600,2010-05-17 07:15:00+00:00,a94060,NaN,usually people just keep op on,9
1971333,45-132,2010-07-25 06:29:00+00:00,jrib,aeroba,"it's not a command, you just open a web browse...",10


In [21]:
# Assigning message ids to allow join with the data  
topics_df['message_id'] = range(1, len(topics_df) + 1)
topics_df.head(10)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,message_id
0,0.798640,0.050402,0.050108,0.050449,0.050402,1
1,0.884184,0.029127,0.028851,0.028789,0.029049,2
2,0.796821,0.050792,0.051126,0.050676,0.050586,3
3,0.200000,0.200000,0.200000,0.200000,0.200000,4
4,0.028947,0.028735,0.884429,0.028753,0.029135,5
5,0.020118,0.919359,0.020144,0.020097,0.020283,6
6,0.597772,0.100399,0.100429,0.100487,0.100913,7
7,0.100946,0.100471,0.101304,0.101999,0.595280,8
8,0.919096,0.020256,0.020144,0.020235,0.020268,9
9,0.006483,0.006779,0.006543,0.006528,0.973667,10


In [22]:
#original dataset extended with topic columns
data_sample = pd.merge(data_sample, topics_df, on='message_id')
data_sample.head(10)

,conversation_id,datetime,from,to,text,message_id,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
0,8-46142,2010-10-25 02:11:00+00:00,Kai696,NaN,anyone spare a moment ^^ ?,1,0.798640,0.050402,0.050108,0.050449,0.050402
1,6-31675,2011-07-17 19:49:00+00:00,TheFuzzball,NaN,macs That should be installed by default...,2,0.884184,0.029127,0.028851,0.028789,0.029049
2,5-45215,2010-09-03 08:54:00+00:00,chris_osx,nightshade,why are you asking that on #ubuntu?,3,0.796821,0.050792,0.051126,0.050676,0.050586
3,22-1223,2010-08-11 04:47:00+00:00,guest99,profxavier,it is.,4,0.200000,0.200000,0.200000,0.200000,0.200000
4,21-2133,2010-01-07 22:33:00+00:00,Deathspike,Mortal_,"No, the tool doesnt even exist on my system.",5,0.028947,0.028735,0.884429,0.028753,0.029135
5,7-9525,2011-04-23 23:40:00+00:00,ceo,erUSUL,am need install inventory.bin,6,0.020118,0.919359,0.020144,0.020097,0.020283
6,3-78839,2010-04-15 01:14:00+00:00,MP1,NaN,Hello,7,0.597772,0.100399,0.100429,0.100487,0.100913
7,51-114,2010-09-03 05:35:00+00:00,undecim,skjoedt,no problem,8,0.100946,0.100471,0.101304,0.101999,0.595280
8,9-11600,2010-05-17 07:15:00+00:00,a94060,NaN,usually people just keep op on,9,0.919096,0.020256,0.020144,0.020235,0.020268
9,45-132,2010-07-25 06:29:00+00:00,jrib,aeroba,"it's not a command, you just open a web browse...",10,0.006483,0.006779,0.006543,0.006528,0.973667


#### Choosing n agents for a given topic 

In [23]:
# View top texts for a topic
data_sample.sort_values(by=['Topic 4'], ascending=False)

,conversation_id,datetime,from,to,text,message_id,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
3059,9-7669,2011-01-05 08:24:00+00:00,plurrtard,NaN,"I'm back, and still from a LiveUSB. Perhaps I ...",3060,0.001753,0.001755,0.001758,0.992965,0.001770
1534,3-52564,2010-04-18 05:06:00+00:00,Perceptron,NaN,"Hey guys, So I want to install Ubuntu 9.10 GNO...",1535,0.001802,0.001807,0.001801,0.992788,0.001801
932,12-4588,2011-01-27 02:36:00+00:00,boiiii,NaN,"hello, i'm trying to set wine utorrent to auto...",933,0.001853,0.001847,0.001844,0.992609,0.001847
1631,7-15844,2010-09-17 14:07:00+00:00,jablo,NaN,Hi all. I have problems booting my ubuntu 10.0...,1632,0.001859,0.001852,0.001857,0.992578,0.001855
6888,5-8127,2011-08-31 00:32:00+00:00,Osmond,NaN,Is Unity incompatible with Radeon video cards?...,6889,0.002077,0.002077,0.002085,0.991678,0.002083
5235,13-3389,2011-05-19 04:16:00+00:00,KM0201,fredhami,"apparently i'm not being clear, or i'm not und...",5236,0.002294,0.002292,0.002286,0.990835,0.002293
5989,3-328332,2010-07-27 01:34:00+00:00,frankbro,NaN,"Ok, we'll wow runs fine. The only problem I ha...",5990,0.002544,0.002561,0.002560,0.989770,0.002565
4413,3-78179,2010-07-28 23:31:00+00:00,Quiet_guy,NaN,Let me try this again with more information. ...,4414,0.002574,0.002568,0.002566,0.989730,0.002562
1993,3-26398,2011-01-26 10:25:00+00:00,hjkl,NaN,"Hey, guys. I'm having a hell of a time trying...",1994,0.002647,0.002643,0.002651,0.989413,0.002647
6972,6-25824,2010-01-03 08:38:00+00:00,cencalrigger,DrManhattan,"Right on. Yes, the truth is unpleasent, but b...",6973,0.002768,0.002759,0.002775,0.988934,0.002764


In [24]:
# coverage of a given topic by different users across all messages
topic = 'Topic 1'
data_sample_topic = data_sample[['from', topic]].groupby(['from'])\
                    .sum().sort_values(by=[topic], ascending=False)

data_sample_topic.head(10)

,Topic 1
from,
ActionParsnip,26.002078
ikonia,23.896123
Dr_Willis,22.698700
bazhang,12.437806
edbian,12.177401
DasEi,11.906684
erUSUL,11.717182
jrib,11.345616
theadmin,8.728770
